In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from huggingface_hub import login
import torch

# Step 1: Login to Hugging Face
HUGGINGFACE_TOKEN = "HF-Token"  # 🔒 Replace with your actual token
login(token=HUGGINGFACE_TOKEN)

# Step 2: Load tokenizer and model
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

print("Loading model (this may take a minute)...")
tokenizer = AutoTokenizer.from_pretrained(model_name, token=HUGGINGFACE_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=HUGGINGFACE_TOKEN,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Step 3: Create generation pipeline
chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Step 4: Prompt engineering
def generate_prompt(user_question):
    return (
        f"You are a friendly and cautious medical assistant. "
        f"Provide clear, general health information. "
        f"Never offer diagnoses or medical treatments. "
        f"Question: \"{user_question}\"\n"
        f"Answer:"
    )

# Step 5: Basic safety filter
def is_safe_question(question):
    unsafe_keywords = [
        "dosage", "how much", "prescribe", "take", "treat", "cure", "medicine for",
        "should I take", "can I take", "pill", "tablet", "capsule", "mg", "ml", "injection"
    ]
    return not any(keyword.lower() in question.lower() for keyword in unsafe_keywords)

# Step 6: Chat interface
def health_chatbot():
    print("🩺 Health Chatbot — powered by Mistral-7B-Instruct")
    print("Type 'exit' to quit.\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Bot: Stay safe and consult a doctor for any medical concerns.")
            break

        if not is_safe_question(user_input):
            print("Bot: I'm sorry, I can't give specific medical or dosage advice. Please consult a doctor.")
            continue

        prompt = generate_prompt(user_input)
        response = chatbot(prompt, max_new_tokens=100, temperature=0.7, do_sample=True)[0]["generated_text"]

        # Extract and clean the answer
        answer = response.split("Answer:")[-1].strip()
        print(f"Bot: {answer}\n")

# Run the chatbot
health_chatbot()


Loading model (this may take a minute)...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


🩺 Health Chatbot — powered by Mistral-7B-Instruct
Type 'exit' to quit.

You: what causes a sore throat?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Bot: A sore throat can be caused by a variety of factors such as viral infections like the common cold or flu, bacterial infections, tonsillitis, strep throat, or even by irritants like smoking or pollution. It's always a good idea to see a healthcare professional if your sore throat persists or is severe.

You: How much paracetamol should I give a child?
Bot: I'm sorry, I can't give specific medical or dosage advice. Please consult a doctor.
You: exit
Bot: Stay safe and consult a doctor for any medical concerns.
